# SSS pre-processing v1.0.0

This notebook addresses an issue with some sound source separation (SSS) routines in which the output files are truncated a few samples with each iteration, by appending silence samples as a buffer. A separate notebook (SSS post-processing) shall truncate the files to restore them to the original number of samples.

## Dependencies management

SoundFile, MoviePy and NumPy packages are installed and imported. OS, Wave, SoundFile, AudioFileClip and NumPy packages are imported. Google Drive is also imported.

In [16]:
# Install necessary dependencies
!pip install soundfile moviepy numpy

# Import required libraries
import os
import wave
import soundfile as sf
from moviepy.editor import AudioFileClip
import numpy as np
import csv
from google.colab import drive

## Configuration

The notebook will process every audio file in the input folder and the results will be saved in the output folder. In this cell, the input and output folders shall be specified. Please adjust the paths as needed. By default, Google Drive is mounted.

In [17]:
# Mount Google Drive, delete if no needed.
drive.mount('/content/drive')

# Configure input and output folders
input_folder = '/content/drive/MyDrive/SSS_tools/pre_input'  # Replace with your input folder
output_folder = '/content/drive/MyDrive/SSS_tools/pre_output'  # Replace with your output folder

# Ensure the output folder exists
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

print(f"Input folder: {input_folder}")
print(f"Output folder: {output_folder}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Input folder: /content/drive/MyDrive/SSS_processing/pre_input
Output folder: /content/drive/MyDrive/SSS_processing/pre_output


# Sample data extraction

Defines a function to extract the number of samples and sample rate of an audio file.

In [18]:
def get_audio_samples(file_path):
    # Check file extension to decide the method of reading
    if file_path.endswith((".wav", ".aiff", ".aif")):
        with wave.open(file_path, 'rb') as audio_file:
            num_samples = audio_file.getnframes()  # Get original number of samples
            sample_rate = audio_file.getframerate()  # Get sample rate
    elif file_path.endswith(".flac"):
        data, sample_rate = sf.read(file_path)  # Get sample rate
        num_samples = len(data)  # Get original number of samples
    elif file_path.endswith((".mp4", ".aac", ".mp3", ".mka", ".mkv")):
        clip = AudioFileClip(file_path)
        data = clip.to_soundarray()
        num_samples = len(data)  # Get original number of samples
        sample_rate = clip.fps  # Get sample rate
        clip.close()
    else:
        raise ValueError("Unsupported file format")

    return num_samples, sample_rate # Return both values

# Silence appending

Defines a function to add 4000 samples of silence to the end of an audio file  as a buffer.

In [19]:
def add_silence(data, silence_samples=4000):
    # Create silence array with 4000 samples
    silence = np.zeros((silence_samples, data.shape[1] if data.ndim > 1 else 1))

    # Append silence to the original audio
    new_data = np.vstack((data, silence))

    return new_data

# Execution

Execution of the above functions requires files with audio content as Wave (.wav), AIFF (.aiff or .aiff), FLAC (.flac), MP3 (.mp3) or AAC (.aac) formats in the input folder.

Results are saved in the output folder. Audio files are saved in FLAC format. Data of original number of samples are saved as CSV files (.csv), one for each input image.

In [20]:
def process_audio_folder(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for file_name in os.listdir(input_folder):
        if file_name.endswith((".wav", ".aiff", ".aif", ".flac", ".mp3", ".aac", ".mp4", ".mka", ".mkv")):
            # Step 1: Get the full input path, the number of samples and sample rate
            input_path = os.path.join(input_folder, file_name)
            num_samples, sample_rate = get_audio_samples(input_path)

            # Step 2: Load the audio data
            if input_path.endswith((".wav", ".aiff", ".aif", ".flac")):
                data, samplerate = sf.read(input_path)
            elif input_path.endswith((".mp3", ".aac", ".mp4", ".mka", ".mkv")):
                clip = AudioFileClip(input_path)
                data = clip.to_soundarray()
                samplerate = clip.fps
                clip.close()
            else:
                raise ValueError("Unsupported file format")

            # Step 3: Add silence to the audio data
            new_data = add_silence(data)

            # Step 4: Save the new audio to the output folder with the same file name and in FLAC format
            output_file_name = os.path.splitext(file_name)[0] + ".flac"
            sf.write(os.path.join(output_folder, output_file_name), new_data, samplerate)

            # Step 5: Save the number of samples and sample rate to a CSV file
            csv_file = os.path.join(output_folder, os.path.splitext(file_name)[0] + ".csv")
            with open(csv_file, mode='w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(["Original Samples", "Sample Rate"])
                writer.writerow([num_samples, sample_rate]) # Write both values

            # Print the number of samples and sample rate for each processed file
            print(f"File: {file_name}, Original Samples: {num_samples}, Sample Rate: {sample_rate}")
            print(f"Added {4000} samples of silence to {file_name}")

# Example usage (optional):
process_audio_folder(input_folder, output_folder)

File: 19. Pequeña Desviación En La Conducta De Los Reyes Magos_000.flac, Original Samples: 6290415, Sample Rate: 44100
Added 4000 samples of silence to 19. Pequeña Desviación En La Conducta De Los Reyes Magos_000.flac


## Version history

v1.0.0 Initial release on 2024-09-21.

## To do

-

## Author information

Wilki Amieva

wilki@hgprods.com.ar.